In [1]:
import sys
if '..' not in sys.path: sys.path.append('..')

In [2]:
from datasets import load_dataset

ds = load_dataset('openai/gsm8k', 'main')

/usr/local/anaconda3/envs/evo/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds['train']['answer']

['Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72',
 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10',
 "In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.\nBetty's grandparents gave her 15 * 2 = $<<15*2=30>>30.\nThis means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.\n#### 5",
 'Maila read 12 x 2 = <<12*2=24>>24 pages today.\nSo she was able to read a total of 12 + 24 = <<12+24=36>>36 pages since yesterday.\nThere are 120 - 36 = <<120-36=84>>84 pages left to be read.\nSince she wants to read half of the remaining pages tomorrow, then she should read 84/2 = <<84/2=42>>42 pages.\n#### 42',
 'He writes each friend 3*2=<<3*2=6>>6 pages a week\nSo he writes 6*2=<<6*2=12>>12 pages every week\nThat means he writes 12*52=<<12*52=624>>624 pages a year\n#### 624',
 "There are 80/100 * 10 = <<80/100*10=8>>8 more purple f

In [ ]:
from utils import BT
from tqdm import tqdm

q_len = []
a_len = []

for thing in tqdm(ds['train']['question']):
    q_len.append(len(BT(thing)['input_ids']))
    
for thing in tqdm(ds['train']['answer']):
    a_len.append(len(BT(thing)['input_ids']))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(q_len)
plt.hist(a_len)
plt.hist([q+a for q, a in zip(q_len, a_len)])

# context length 512 will fit

In [ ]:
import re
import json
from tqdm import tqdm

def aux(q, a):
    a = re.sub(r'<<.*?>>', '', a)
    a_toks = BT(a, max_length=512, truncation=True)['input_ids'][1:-1]
    a_traj = []
    
    while a_toks:
        a_traj.append(BT.decode(a_toks[:]))
        a_toks = a_toks[:len(a_toks)//2]
    
    return [q] + [f'{q} {a}' for a in a_traj][::-1]

train_val_split = ds['train'].train_test_split(test_size=0.1)
train = train_val_split['train']
val = train_val_split['test']

with open('../data/gsm/gsm_train_1.jsonl', 'w') as f:
    for x in tqdm(train):
        q = x['question']
        a = x['answer']
        json.dump([aux(x['question'], x['answer']), 0], f)
        f.write('\n')
        
with open('../data/gsm/gsm_dev_1.jsonl', 'w') as f:
    for x in tqdm(val):
        q = x['question']
        a = x['answer']
        json.dump([aux(x['question'], x['answer']), 0], f)
        f.write('\n')

In [ ]:
from utils import generate_prefix_alignment

generate_prefix_alignment(
    input_path='../data/gsm/gsm_train_1.jsonl',
    output_dir='../data/gsm/cache_1',
    cache_prefix='sup-gsm-1',
    max_len=512
)